In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\SymbDigi
data = pd.read_csv('SymbDigiCateg_Methylome_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\SymbDigi


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,Symbol_Digit,cg20815302,cg25795414,cg00959375,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,0,200991620021_R01C01,3001,2,1,1,1,0.907957,0.86891,0.752844,...,0.880496,0.928546,0.900567,0.8778,0.836823,0.216268,0.23435,0.68155,0.103758,0.83789


In [9]:
X = data.drop(['Unnamed: 0','Sentrix','PATNO','HYS','Gender','Symbol_Digit'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [10]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [11]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7258064516129032

In [12]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100


BorutaPy finished running.

Iteration: 	33 / 100
Confirmed: 	2
Tentative: 	0
Rejected: 	192581


BorutaPy(estimator=RandomForestClassifier(n_estimators=20,
                                          random_state=RandomState(MT19937) at 0x16F81D2EC40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x16F81D2EC40, verbose=1)

In [13]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [14]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [15]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,APPRDX,1
7938,cg27171575,1
187085,cg15419852,2


In [16]:
Top50.to_csv ('SymbDigi_BorutaMale_Top50.csv', index = False)
selected_rf_features.to_csv ('SymbDigi_BorutaMale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
